# LN - Data PP - Stability and efficiency calculations

## Import libraries and data

In [1]:
import sqlite3
import numpy as np
import pandas as pd
import networkx as nx
import itertools
#import matplotlib.pyplot as plt
import time
import pickle

import os
import re
import sys
import io
from itertools import islice
import math

from tqdm.notebook import trange, tqdm
from time import sleep

from dask_cloudprovider import FargateCluster
from dask.distributed import Client
import dask.array as da
import dask
dask.config.set({'distributed.scheduler.allowed-failures': 50}) 


import boto3


In [2]:


# Load Data

# Initiate s3 resource

session = boto3.session.Session()
s3 = session.resource('s3')
bucket='ln-strategy-data'
extraction_id=1585344554



# Dataframe

decisions_load = s3.Object(bucket_name=bucket, key='LN_channels.csv').get()
decisions_df=pd.read_csv(io.BytesIO(decisions_load['Body'].read()))

# Channel closures
closure_file = s3.Object(bucket_name=bucket, key='channel_closures.p').get()
channel_closures = pickle.loads(closure_file['Body'].read())
    
    
# Channel openings 
opens_file = s3.Object(bucket_name=bucket, key='channel_opens.p').get()
channel_opens = pickle.loads(opens_file['Body'].read())

    

# Create list with graph keys

#TODO: Save graphs as numpy array in single H5 file to reduce. Test if creating graphs takes longer than reading from S3

# graph_dir='./data/graph_snapshots' - For local tests

graph_keys = [obj.key 
        for obj in s3.Bucket(name=bucket).objects.all()
        if re.match(".*"+str(extraction_id)+"_connected/.*\.gpickle",obj.key)]





In [3]:
# Test: extracted formats
print("Number of graph keys:{}".format(len(graph_keys)))
print("---Sample graph keys---")
print(graph_keys[0])
print("---Sample channel opens---")
print(channel_opens[513675])
print("---Sample channel closures---")
print(channel_closures[592638])



Number of graph keys:3996
---Sample graph keys---
graph_snapshots/1585344554_connected/505149.gpickle
---Sample channel opens---
[(5474, 7365, {'capacity': 50000, 'open_fee': 306, 'dec_id': 1, 'channel_id': '513675x2245x0'})]
---Sample channel closures---
[(3098, 1492, {'close_type': 'mutual', 'dec_id': 0, 'channel_id': '505149x622x0'}), (2104, 3098, {'close_type': 'force', 'dec_id': 26620, 'channel_id': '570913x720x1'})]


In [5]:
decisions_df.sort_values(by=['open_block'],inplace=True,ascending=True)
decisions_df.head(10)

,short_channel_id,open_block,open_transaction,address,close_block,close_transaction,node0,node1,satoshis,last_seen,...,close_time,close_fee,last_update,close_type,close_htlc_count,close_balance_a,close_balance_b,dec_id,node0_id,node1_id
0,505149x622x0,505149,f6bc767df9148ebf76d2b9baf4eb46e3230712c2bf5a51...,bc1qjmg6ev344fenh3zhg0yjl6hyvxpxluw6x9nn2a5lv4...,592638.0,82cb2ea2a06c8c453d8b9ca08e17bbefe87225aa380b2d...,0250373555232cec757ea141273e75381c84cc3ab22f1e...,02ef61a252f9504a42fc264a28476f44cea0711a44b2da...,300000,2019-08-22 02:49:00,...,1.567276e+09,184.0,1.563172e+09,mutual,0.0,3570.0,296246.0,0,3098,1492
38787,506402x1391x1,506402,2cdfc4fec2049d66a04fa5bdf468efb19c0354c60b8cf2...,bc1qvjx5t8y7j83udzuj38ukmqecv5d9jn762mchxkgvaf...,NaN,NaN,0313f9449cdb528dc9707c02da507cc9306eedc415091c...,035f1498c929d4cefba4701ae36a554691f526ff60b176...,1111934,2019-07-08 03:36:43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38787,5407,5432
38788,506847x1633x0,506847,19ee11ce977facd380b92126834a3aca318f3cb905d99b...,bc1q29g43xrz9gujgt60gykzq3vh0ewfav7vmfqcnmf50u...,NaN,NaN,023d280ae29f84dcfd289eb66b57227fea3a7bde97ec28...,0273081ce642554d5a68a5236564fe88a3783457dc09e5...,40000,2019-07-08 03:36:43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38788,983,4911
38789,508075x1694x1,508075,e267e54872053a7618567f31a9d27e38cdbff0e4176144...,bc1qpxzqp2xyy0gzn8xwu6lqg3a66tuhsg5w849t0j5rdr...,NaN,NaN,03557fd11b58cb93d2ad4fab4dd4cff7462a97e21e8f6b...,03cbf298b068300be33f06c947b9d3f00a0f0e8089da32...,100000,2019-07-08 03:36:43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38789,6779,2471
58766,508090x1515x1,508090,33d645657de8a587137b8039e52452557d4279a3f47366...,bc1qneudwey0dpgy9nj2g8ech0lqqrhz52agcj984rs6zh...,616838.0,123777e4dfadc7c008a54c2d55b670067a58cdcbc8b2ec...,028314f021602092779aedd4ef39f3b5809f9b6046f8bc...,02d4531a2f2e6e5a9033d37d548cff4834a3898e74c3ab...,400000,2020-02-10 03:00:34,...,1.581368e+09,4887.0,1.581152e+09,NaN,NaN,NaN,NaN,58766,7635,6677
38790,508320x2072x0,508320,40944505d8e1a55e6923283cee629757500aa344142650...,bc1qcrsrc0xzt3uumh3uyftn600ku9vmamth50zdzfjpv2...,NaN,NaN,0347ffcb271ef54fa6103fc6392370ce398d1bc9846082...,034a5fdb2df3ce1bfd2c2aca205ce9cfeef1a5f4af21b0...,1050,2019-07-08 03:36:44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38790,3325,5357
38791,508400x1699x0,508400,71c790031c4b99593848dde6054366d0df2ad85c2806b7...,bc1qc4r2cdzucu2xncqv8sde97c0kumvqlrv9ehmfxykzu...,NaN,NaN,0268cbff355499bf97387dec397614ab3bd0e3e4361475...,034a5fdb2df3ce1bfd2c2aca205ce9cfeef1a5f4af21b0...,1050,2019-07-08 03:36:44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38791,1047,5357
38793,508447x656x0,508447,5f741522f787c5fde67f2fd73b220e9af6aff9e4ee700f...,bc1qe7uvtpg6klu0awfq479xz56pgm9mh9d67k7m3f6hc2...,NaN,NaN,02952cc6b7f92226e490d6354636604a26484a396627c9...,0333b58f86a0c31da7ee61a4bc2232faccf8c4fe4167fe...,50000,2019-07-08 03:36:44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38793,1538,5395
38794,508447x659x0,508447,abc55727b4d5cfa39d84e48df410230d9c1b100b3cc07a...,bc1qhacv3gscv40muy3u57sr9msrqspa2lcdxrhehm04ak...,NaN,NaN,023e2b9bb357b1bf135d59551418178ef020cdd77efee7...,0333b58f86a0c31da7ee61a4bc2232faccf8c4fe4167fe...,50000,2019-07-08 03:36:44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38794,6244,5395
38795,508447x661x0,508447,b592861b271aedadf6ad529b158df0e962b830e2b6056b...,bc1qxq49306phnepteaxx98gxx9tfjxw35umasvvep3w0k...,NaN,NaN,023c5f2b69161742cc49da5be6cb7c940465fc608ff216...,0333b58f86a0c31da7ee61a4bc2232faccf8c4fe4167fe...,50000,2019-07-08 03:36:44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38795,4983,5395


In [6]:
sys.getsizeof(decisions_df)

63662158

## Connection to AWS - Fargate Clusters

In [7]:
cluster = FargateCluster(n_workers=20,scheduler_timeout='60 minutes',image='dsrincon/dask-graph:nx-scipy-v1')

/opt/conda/lib/python3.7/contextlib.py:119: UserWarning: Creating your cluster is taking a surprisingly long time. This is likely due to pending resources on AWS. Hang tight! 
  next(self.gen)
/opt/conda/lib/python3.7/site-packages/dask_cloudprovider/providers/aws/ecs.py:321: RuntimeWarning: get_log_events rate limit exceeded, retrying after delay.
  RuntimeWarning,


In [8]:
cluster

In [9]:
#client = Client(cluster)
cluster=Client(cluster)

/opt/conda/lib/python3.7/site-packages/distributed/client.py:1079: VersionMismatchWarning: Mismatched versions found

python
+---------------------------+---------------+
|                           | version       |
+---------------------------+---------------+
| client                    | 3.7.3.final.0 |
| scheduler                 | 3.7.4.final.0 |
| tcp://172.31.10.160:44217 | 3.7.4.final.0 |
| tcp://172.31.18.164:32951 | 3.7.4.final.0 |
| tcp://172.31.2.251:36623  | 3.7.4.final.0 |
| tcp://172.31.22.249:38817 | 3.7.4.final.0 |
| tcp://172.31.26.57:37691  | 3.7.4.final.0 |
| tcp://172.31.30.200:44571 | 3.7.4.final.0 |
| tcp://172.31.40.252:36887 | 3.7.4.final.0 |
| tcp://172.31.44.187:38427 | 3.7.4.final.0 |
| tcp://172.31.46.55:46269  | 3.7.4.final.0 |
| tcp://172.31.52.48:34913  | 3.7.4.final.0 |
| tcp://172.31.56.35:34207  | 3.7.4.final.0 |
| tcp://172.31.58.131:38549 | 3.7.4.final.0 |
| tcp://172.31.64.23:36965  | 3.7.4.final.0 |
| tcp://172.31.69.113:46531 | 3.7.4.final.0 |
|

## Defining Utilities

### Saving

**FUNCTION: Write output to DataFrame**

In [23]:
# Function write output to DataFrame

"""
Function
--------
add_columns
    Function that takes an output from a decision comparisson computation and adds it's results for nodes 1 and 0 in the main DataFrame

Parameters
----------
output_diclist : list
    Dictionary of the form (node0_dic_i,node1_dic_i) where i runs for all of the blocks being compared. 

original_df: Pandas DataFrame
    Original DataFrame containing the opening and closure information for each channel, with a column named 'short_channel_id' to denote 
    id of channel. 

column_name_node0: string
    Name for column in dataframe where the results will be stored for node 0
    
column_name_node1: string
    Name for column in dataframe where the results will be stored for node 1

    
Returns
-------
no_changes: list
    List with the 'short_channel_id' of the channels edited. 
 
"""


def add_columns(output_diclist,original_df,column_name_node0,column_name_node1):


    # Merge individual dictionaries into one for each node
    node0_dic={}
    node1_dic={}
    for dic_tuple in output_diclist:
        node0_dic.update(dic_tuple[0])
        node1_dic.update(dic_tuple[1])
    
    # Add to DataFrame

    # Create empty columns
    original_df[column_name_node0]=np.nan
    original_df[column_name_node1]=np.nan

    # Populate df with values
    original_df[column_name_node0]=original_df['short_channel_id'].map(node0_dic)
    original_df[column_name_node1]=original_df['short_channel_id'].map(node1_dic)
    
    # Calculate values changed
    rows_edited=(original_df[original_df[column_name_node0].notnull()]['short_channel_id']).tolist()
    
    return rows_edited


**FUNCTION: Save python object to S3 using pickle**

In [24]:
# Function to write pickle data to S3 bucket

"""
Function
--------
pickle_save_s3
    Saves Python object to S3 as pickle object

Parameters
----------
obj : <any>
    Python Object

blocks: list
    List of extracted blocks

extraction_id: int
    Number of block extraction
    
name: string
    Name of object to add to filename in S3

    
Returns
-------
response: int
    HTTP response code from S3 
 
"""


def pickle_save_s3(obj,blocks,extraction_id,name):


    # SAVE betweeness dictionary to disk
    start_block=np.min(np.array(blocks))
    end_block=np.max(np.array(blocks))
    no_blocks=len(blocks)

    # Load S3 and bucket details
    session = boto3.session.Session()
    s3 = session.resource('s3')

    # File path and name ([extraction_id][name]-[no_blocks]-[start_block]-[end_block])
    key_snapshot_bet='graph_snapshots/'+str(extraction_id)+'_connected/.data_transformations/'+str(extraction_id)+name+'-'+str(no_blocks)+'-'+str(start_block)+'-'+str(end_block)+'.pkl'

    # Create pickle object and send to S3
    pickle_byte_obj = pickle.dumps(obj) 
    response=s3.Object(bucket,key_snapshot_bet).put(Body=pickle_byte_obj)
    
    return response['ResponseMetadata']

## Load Data

### Graphs

**FUNCTION: Load single graph**

In [27]:


"""
Function
--------
load_graph
    Loads networkX (pickle serialized) object from S3

Parameters
----------
key : str
    Path in S3 bucket for individual pickled serialized networkX graph object 

    
Returns
-------
response: networkX graph
    Graph object
 
"""

def load_graph(key):
    session = boto3.session.Session()
    s3 = session.resource('s3')
    response = s3.Object(bucket_name=bucket, key=key).get()
    G=pickle.loads(response['Body'].read())
    
    return G
   

**SCRIPT: Create network snapshots as a list of delayed graph objects and list of blocks**

In [28]:
# Base lists to be populated
graph_snapshots=[]
blocks=[]

for key in graph_keys[700:900]: # Change to [700:] for full range
    
    # Create block list from file_names
    block_i=int(key.split(".")[0].split("/")[-1]) 
    blocks.append(block_i)
    
    # Extract graphs
    G=dask.delayed(load_graph)(key)
    graph_snapshots.append(G)

**TESTS**

In [ ]:
# Test Lazy Graph extract
blocks_att=[]
for i in range(len(graph_snapshots)):
    graph_i=dask.compute(graph_snapshots[i])
    block=graph_i.graph['block']
    blocks_att.append(block)

print(blocks_att)

#graph_snapshots=dask.compute(*graph_snapshots)
#block=graph_snapshots[0].graph['block']
    
#print(len(graph_snapshots[5]))
#print(graph_snapshots[3].graph['block'])

# Delayed testing
#results = dask.compute(*futures)
#graphs=dask.compute(*graph_snapshots)


# Comparative Analysis

In order to understand the potential motivations behind each decision we analyze each decission (opening or closure of a channel) independently from the perspective of each of the participants in the decission, which we'll call the node under analysis. For each decission we extract or compute the following information: 

Betweenness centrality measures how central is a network to the flow of information in a network. In the case of the Lightning Network the higher the betweenness centrality of a node, the more transactions (messages) that are routed through it. In particular, we will use a measure of betweenness centrality defined in (Brandes and Fleischer 2005 - https://link.springer.com/chapter/10.1007/978-3-540-31856-9_44) that models infomation through a network, as electric current, efficiently and not only considering shortest path. This allows us to account for the fact that not all transactions travel through shortes path given that there are fee and capacity considerations.  

## Baseline Measurments

**FUNCTION: Property measurement for a collection of graphs**

In [35]:
# Function to calculate base measurement for every graph in snapshot

"""
Function
--------
collection_measure
    Iterates over graph snapshots and calculates measurement for every node in each of the graphs.

Parameters
----------
g_snapshots : list
    List of delayed nx graph elements contianing graph snapshots
    

measurement: string
    Name of the type of measurment that will applied to the graph. (See graph_measurement function for options)



Returns
-------
snapshot_mes_dic: dic
    Dictionary with blocks as keys and dic of measurements as values

"""
def collection_measure(g_snapshots,measurement):
    
    snapshot_mes_list=[]
   
    for g in graph_snapshots:
      
        
        measurement_input=(g,measurement,'capacity')
        
        b_g_tuple=dask.delayed(graph_measurement)(measurement_input)
        snapshot_mes_list.append(b_g_tuple)
        

    #futures = dask.persist(*snapshot_mes_list)
    
    start=time.time()
    #snapshot_mes_list = dask.compute(*futures)
    snapshot_mes_list = dask.compute(*snapshot_mes_list)
    snapshot_mes_dic={record[0]:record[1] for record in snapshot_mes_list}
    end=time.time()
    print('Compute in seconds: {}'.format(end-start))
    
    return snapshot_mes_dic

**FUNCTION: Property measurment for a single graph**

In [29]:

"""
Function
--------
graph_measurement
    Performs specific graph measurement 

Parameters
----------
measurment_input: tuple
    g : nx graph
        NetworkX graph object over which measurment will be calculated for each node

    measurment: string
        Type of measurement to be performed on graph
        
    weight: string
        Edge attribute to be used as weight
    
Returns
-------
g_dir: dir
    Dictionary with measurment values for each node
"""

def graph_measurement(measurment_input):
    
    g=measurment_input[0]
    measurement=measurment_input[1]
    weight=measurment_input[2]
    
    if measurement=='current_betweeness':
        g_dir=nx.algorithms.centrality.current_flow_betweenness_centrality(g,weight=weight)
    
    elif measurement=='current_closeness':
        g_dir=nx.algorithms.centrality.current_flow_closeness_centrality(g,weight=weight)
    
    block=g.graph['block']
    
    return (block,g_dir)

**FUNCTION: Property measurment for a couple of nodes in a graph**

In [40]:

"""
Function
--------
node_measurement
    Performs selected graph measurment on specific nodes in graph

Parameters
----------
g : nx graph
    NetworkX graph object over which the measurment will be performed

measurement: string
    Type of measurement to be performend in graph
    
node0: int
    Node id for node 0

node1: int
    Node id for node 1
    
Returns
-------
node_tuple: tuple
    Tuple of the form (node0_mes,node1_mes)
    
    node0_mes: float
        Graph measurement for node0
    node1_mes: float
        Graph measurement for node1
"""


        
    
    

def node_measurement(g,measurement,node0,node1):
    
    measurement_input=(g,measurement,'capacity')
    block,g_dir=graph_measurement(measurement_input)
    
    node0_mes=g_dir[node0]
    node1_mes=g_dir[node1]
        
    # Update marginal values for node0 and node1
        
    if (g.has_node(node0)): #If connected component of marginal graph contains node0 find betweeness
        node0_mes=g_dir[node0]
    else: # else update with fixed value
        node0_mes=0
            
    if (g.has_node(node1)): #If connected component of marginal graph contains node1 find betweeness
        node1_mes=g_dir[node1]
    else: # else update with fixed value
        node1_mes=0
    
    return (node0_mes,node1_mes)
    
    

**SCRIPT: Baseline Current Betweeness**

In [36]:
# Calculate Baseline betweeness
snapshot_bet=collection_measure(graph_snapshots,'current_betweeness')

Compute in seconds: 61.81248641014099


In [19]:
# Save baseline current betweeness to S3
response=pickle_save_s3(snapshot_bet,blocks,extraction_id,'snapshot_bet')
if response['HTTPStatusCode']==200:
    print('Save to S3 succesful')
else:
    print(response)

Save to S3 succesful


**SCRIPT: Baseline Current Closeness**

In [37]:
# Calculate Baseline current closeness
snapshot_clo=collection_measure(graph_snapshots,'current_closeness')

Compute in seconds: 9.97524118423462


In [22]:
# Save baseline current closeness to S3
response=pickle_save_s3(snapshot_clo,blocks,extraction_id,'snapshot_clo')
if response['HTTPStatusCode']==200:
    print('Save to S3 succesful')
else:
    print(response)

Save to S3 succesful


**TESTS**

In [419]:
# Test Save
session = boto3.session.Session()
s3 = session.resource('s3')
snapshot_bet_file = s3.Object(bucket_name=bucket, key=key_snapshot_bet).get()
snapshot_bet_test = pickle.loads(snapshot_bet_file['Body'].read())
snapshot_bet_test==snapshot_bet

True

## Comparissons

**FUNCTION: Compare property changes for nodes in a collection of graphs**

In [33]:

"""
Function
--------
collection_compare
    Iterates over blocks to calculate marginal change in metric for nodes that made decisions (opens/closures)

Parameters
----------
blocks : list
    List of blocks extracted when reading graphs
    
    
dec_dic: dic
    Dictionary with channel decisions (open or closure) per block
    
    
graph_snapshots: list
    List of delayed dask objects each pointing to a graph snapshot to be loaded from S3

snapshots_base: dic
    Dictionary of dictionaries containing the base measurments per block per node

measurement: string
    Name of the type of measurment that will applied to the graph
    
type_dec: string
    The type of decisions that will be analyzed 'opens' or 'closures'
    

Returns
-------
futures: list
    List of tuples of the form (dic_node0,dic_node1) where dic_node0/1 is a dictionary containing the marginal changes for node0/1 
    for every decision in a given block. The dictionaries are future dask objects that still need to be explicitly computed. 

"""
def collection_compare(blocks,dec_dic,graph_snapshots,snapshots_base,measurement,type_dec):

    futures_list=[] # list to populate with futures per block
    
    with tqdm(total=len(range(1,len(graph_snapshots)))) as pbar:
        for i in range(1,len(graph_snapshots)): # iterate through blocks

            # extract information from parameters and construct input tuple to delayed function
            block=blocks[i]
            block_prev=blocks[i-1]
            block_dec=dec_dic[block]
            g=graph_snapshots[i-1] # Pass previous graph
            block_base=snapshots_base[block_prev]
            block_res=snapshots_base[block]
            input_tuple=(block,g,block_dec,block_base,measurement,type_dec,block_res)
            
            # submit to delayed function and append to list
            output_tuple=dask.delayed(graph_compare)(input_tuple)
            futures_list.append(output_tuple)
            pbar.update(1)

    # persist to disk and return
    futures = dask.persist(*futures_list)
    return futures
    

**FUNCTION: Compare property changes for nodes in a graph**

In [38]:

"""
Function
--------
graph_compare
    Calculates marginal change in metric for node0, node1 make decisions (open/close channels) in a single block

Parameters
----------
input_tuple : tuple
    
    block: int
        Block number
    g: nx_graph 
        Graph snapshot (as dask delayed object)
    block_dec: list
        List of tuples in nx edge format (u,v,att_dic) for all the decisions (channel opens/closures) made in that block  
    block_base: dic
        Dictionary of base measurements for each node in the graph snapshot corresponding to that block
    measurement: string
        Name of measurement to be computed
    type_dec: string
    The type of decisions that will be analyzed 'opens' or 'closures'
    
    block_res: dic
        Dictionary of base measurements for each node in the graph snapshot corresponding to the next block
    

Returns
-------
nodes_mar_dic: tuple
    Tuples of the form (mar_node0_dic_i,mar_node0_dic_i) where each element in the tuple is a dictionary containing the marginal changes for node0/1 
    for every node0 and node1 involved in a decision (channel open/closures) in the block.
"""


def graph_compare(input_tuple):
    
    block=input_tuple[0]
    G=input_tuple[1]
    block_dec=input_tuple[2]
    block_base=input_tuple[3]
    measurement=input_tuple[4]
    type_dec=input_tuple[5]
    block_res=input_tuple[6]
   
    mar_node0_dic_i={} # dictionary to story function output
    mar_node1_dic_i={} 
    
    # For each decision calculate marginal change in measure for node0 and node1
    for edge in block_dec:
        
        # Extract info about channel
        channel_id=edge[2]['channel_id']
        node0=edge[0]
        node1=edge[1]

        
        # Copy original graph
        g_mar=G.copy()   
        old_nodes=False
        
        
        
        # Retrieve base measurement before channel if nodes existed, else define base measure as 0
        if (g_mar.has_node(node0)):
            node0_base=block_base[node0]
            old_nodes=True
        else:
            node0_base=0
            
        if (g_mar.has_node(node1)):
            node1_base=block_base[node1]
            old_nodes=True
        else:
            node1_base=0
        
            
        if old_nodes: # If at least one node is old (part of the connected graph)
            
            if type_dec=='mar_opens': # marginal calculation for opens
                
                
                # Define and add edges and calculate betweeness if at least one of the nodes is in graph 
                edge_list=[edge]
                g_mar.add_edges_from(edge_list)
                g_mar_mes=node_measurement(g_mar,measurement,node0,node1)
                
                # Update measurement values after marginal change
                node0_new_mes=g_mar_mes[0]
                node1_new_mes=g_mar_mes[1]
            
            elif type_dec=='mar_closures': # marginal calculation for closes
                
                # Define and remove edges, define new connected graph and calculate betweeness 
                edge_list=[(node0,node1)]
                g_mar.remove_edges_from(edge_list)   
                connected_components=[c for c in nx.algorithms.components.connected_components(g_mar)]
                g_mar=g_mar.subgraph(connected_components[0]).copy() 
                g_mar_mes=node_measurment(g_mar,measurement,node0,node1)
                node0_new_mes=g_mar_mes[0]
                node1_new_mes=g_mar_mes[1]
                
            elif type_dec=='actual': # actual calculation for both opens and closures
                
                # Check individualy if in the graph for the resulting block the node is present (in the connected component, 
                # if not assign measurment to 0. 
                
                try:
                    node0_new_mes=block_res[node0]

                except KeyError:
                    node0_new_mes=0

                try:
                    node1_new_mes=block_res[node1]

                except KeyError:
                    node1_new_mes=0

                
                   
            node0_mar=(node0_new_mes-node0_base)
            node1_mar=(node1_new_mes-node1_base) 
        
        
        else: # If both nodes are new (outside of connected graph) their marginal decision outcome is 0
            node0_mar=0
            node1_mar=0

        
        # Update dictionary - new betweenness
        mar_node0_dic_i[channel_id]=node0_mar
        mar_node1_dic_i[channel_id]=node1_mar
        
    
    return (mar_node0_dic_i,mar_node1_dic_i)

### MARGINAL - CURRENT BETWEENESS

**SCRIPT: Marginal current betweeness for opens**

In [41]:
# Compute marginal betweeness for channel openings

futures_bet_maropen=collection_compare(blocks,channel_opens,graph_snapshots,snapshot_bet,measurement='current_betweeness',type_dec='mar_opens')
start=time.time()
bet_maropen_diclist = dask.compute(*futures_bet_maropen)
end=time.time()
print('Compute in seconds: {}'.format(end-start))
print('Size in memory: {}'.format(sys.getsizeof(bet_maropen_diclist)))




Compute in seconds: 67.94000720977783
Size in memory: 1640


In [42]:
# Write to marginal current betweeness for opens into decisions Dataframe

bet_maropen_channels=add_columns(bet_maropen_diclist,decisions_df,'bet_maropen_node0','bet_maropen_node1')

# Updated DataFrame

print('Rows edited with marginal current betweeness for opens: {}'.format(len(bet_maropen_channels)))
decisions_df[decisions_df['short_channel_id'].isin(bet_maropen_channels)].head(2)


Rows edited with marginal current betweeness for opens: 218


,short_channel_id,open_block,open_transaction,address,close_block,close_transaction,node0,node1,satoshis,last_seen,...,last_update,close_type,close_htlc_count,close_balance_a,close_balance_b,dec_id,node0_id,node1_id,bet_maropen_node0,bet_maropen_node1
290,525952x80x0,525952,9e0ec787af59bdb7affc8be04c379733f94b06e52833fb...,bc1qk9trrgwz8kl6ryklxzmaky2jfgc0lycv7zcajeu82m...,578854.0,00d3ddedfb6cd6662745b912eb361fc70ed57a088c5e26...,024cd3be18617f39cf645851e3ba63f51fc13f0bb09e3b...,028705acfe269be139d536a5c7a05961cbbed3484a2c9b...,117838,2019-06-17 03:46:46,...,NaN,force,0.0,116570.0,0.0,290,2063,6115,0.004865,2.181354e-03
40048,525989x1709x0,525989,af14ba1556970820cd358806adfe3f739ed741dccea0bf...,bc1qaw0nxph7h3kxgng0zg8jftjsulsel35mgy5t7t77x6...,NaN,NaN,02e9fa6b495d07ce19796b06b5c0f51a401dbb48935c3a...,033401a6d1a5ac4ac54a0a45b6f87c871ec897c46b9dc9...,21600,2019-07-08 03:39:18,...,NaN,NaN,NaN,NaN,NaN,40048,4822,5415,0.003510,1.474100e-16


**SCRIPT: Marginal current betweeness for closures**

In [43]:
# Compute marginal betweeness for channel closures

futures_bet_marclose=collection_compare(blocks,channel_closures,graph_snapshots,snapshot_bet,measurement='current_betweeness',type_dec='mar_closures')
start=time.time()
bet_marclose_diclist = dask.compute(*futures_bet_marclose)
end=time.time()
print('Compute in seconds: {}'.format(end-start))
print('Size in memory: {}'.format(sys.getsizeof(bet_marclose_diclist)))



Compute in seconds: 4.222078323364258
Size in memory: 1640


In [44]:
# Write to marginal current betweeness for closures into decisions Dataframe

bet_marclose_channels=add_columns(bet_marclose_diclist,decisions_df,'bet_marclose_node0','bet_marclose_node1')

# Updated DataFrame

print('Rows edited with marginal current betweeness for closures: {}'.format(len(bet_marclose_channels)))
decisions_df[decisions_df['short_channel_id'].isin(bet_marclose_channels)].head(2)

Rows edited with marginal current betweeness for closures: 0


,short_channel_id,open_block,open_transaction,address,close_block,close_transaction,node0,node1,satoshis,last_seen,...,close_htlc_count,close_balance_a,close_balance_b,dec_id,node0_id,node1_id,bet_maropen_node0,bet_maropen_node1,bet_marclose_node0,bet_marclose_node1


### MARGINAL - CURRENT CLOSENESS

**SCRIPT: Marginal current closeness for opens**

In [45]:
# Compute marginal betweeness for channel openings

futures_clo_maropen=collection_compare(blocks,channel_opens,graph_snapshots,snapshot_clo,measurement='current_closeness',type_dec='mar_opens')
start=time.time()
clo_maropen_diclist = dask.compute(*futures_clo_maropen)
end=time.time()
print('Compute in seconds: {}'.format(end-start))
print('Size in memory: {}'.format(sys.getsizeof(clo_maropen_diclist)))



Compute in seconds: 13.059130668640137
Size in memory: 1640


In [46]:
# Write to marginal current betweeness for opens into decisions Dataframe

clo_maropen_channels=add_columns(clo_maropen_diclist,decisions_df,'clo_maropen_node0','clo_maropen_node1')

# Updated DataFrame

print('Rows edited with marginal current closeness for opens: {}'.format(len(clo_maropen_channels)))
decisions_df[decisions_df['short_channel_id'].isin(clo_maropen_channels)].head(2)


Rows edited with marginal current closeness for opens: 218


,short_channel_id,open_block,open_transaction,address,close_block,close_transaction,node0,node1,satoshis,last_seen,...,close_balance_b,dec_id,node0_id,node1_id,bet_maropen_node0,bet_maropen_node1,bet_marclose_node0,bet_marclose_node1,clo_maropen_node0,clo_maropen_node1
290,525952x80x0,525952,9e0ec787af59bdb7affc8be04c379733f94b06e52833fb...,bc1qk9trrgwz8kl6ryklxzmaky2jfgc0lycv7zcajeu82m...,578854.0,00d3ddedfb6cd6662745b912eb361fc70ed57a088c5e26...,024cd3be18617f39cf645851e3ba63f51fc13f0bb09e3b...,028705acfe269be139d536a5c7a05961cbbed3484a2c9b...,117838,2019-06-17 03:46:46,...,0.0,290,2063,6115,0.004865,2.181354e-03,NaN,NaN,0.077226,1.660158
40048,525989x1709x0,525989,af14ba1556970820cd358806adfe3f739ed741dccea0bf...,bc1qaw0nxph7h3kxgng0zg8jftjsulsel35mgy5t7t77x6...,NaN,NaN,02e9fa6b495d07ce19796b06b5c0f51a401dbb48935c3a...,033401a6d1a5ac4ac54a0a45b6f87c871ec897c46b9dc9...,21600,2019-07-08 03:39:18,...,NaN,40048,4822,5415,0.003510,1.474100e-16,NaN,NaN,-0.011045,11.151760


**SCRIPT: Marginal current closeness for closures**

In [47]:
# Compute marginal current closeness for channel closures

futures_clo_marclose=collection_compare(blocks,channel_closures,graph_snapshots,snapshot_clo,measurement='current_closeness',type_dec='mar_closures')
start=time.time()
clo_marclose_diclist = dask.compute(*futures_clo_marclose)
end=time.time()
print('Compute in seconds: {}'.format(end-start))
print('Size in memory: {}'.format(sys.getsizeof(clo_marclose_diclist)))


Compute in seconds: 4.227483749389648
Size in memory: 1640


# Write to marginal current betweeness for opens into decisions Dataframe

clo_marclose_channels=add_columns(clo_marclose_diclist,decisions_df,'clo_marclose_node0','clo_marclose_node1')

# Updated DataFrame

print('Rows edited with marginal current closeness for closures: {}'.format(len(clo_marclose_channels)))
decisions_df[decisions_df['short_channel_id'].isin(clo_marclose_channels)].head()

### ACTUAL - CURRENT BETWEENESS

**SCRIPT: Actual current betweeness for opens**

In [49]:
# Compute actual current betweeness for channel opens

futures_bet_actopen=collection_compare(blocks,channel_opens,graph_snapshots,snapshot_bet,measurement='current_betweeness',type_dec='actual')
start=time.time()
bet_actopen_diclist = dask.compute(*futures_bet_actopen)
end=time.time()
print('Compute in seconds: {}'.format(end-start))
print('Size in memory: {}'.format(sys.getsizeof(bet_actopen_diclist)))


Compute in seconds: 7.372019052505493
Size in memory: 1640


In [59]:
# Write to marginal current betweeness for opens into decisions Dataframe

bet_actopen_channels=add_columns(bet_actopen_diclist,decisions_df,'bet_actopen_node0','bet_actopen_node1')

# Updated DataFrame

print('Rows edited with actual current betweeness for opens: {}'.format(len(bet_actopen_channels)))
decisions_df[decisions_df['short_channel_id'].isin(bet_actopen_channels)].head(2)

Rows edited with actual current betweeness for opens: 218


,short_channel_id,open_block,open_transaction,address,close_block,close_transaction,node0,node1,satoshis,last_seen,...,bet_marclose_node0,bet_marclose_node1,clo_maropen_node0,clo_maropen_node1,clo_marclose_node0,clo_marclose_node1,bet_actopen_node0,bet_actopen_node1,bet_actclose_node0,bet_actclose_node1
290,525952x80x0,525952,9e0ec787af59bdb7affc8be04c379733f94b06e52833fb...,bc1qk9trrgwz8kl6ryklxzmaky2jfgc0lycv7zcajeu82m...,578854.0,00d3ddedfb6cd6662745b912eb361fc70ed57a088c5e26...,024cd3be18617f39cf645851e3ba63f51fc13f0bb09e3b...,028705acfe269be139d536a5c7a05961cbbed3484a2c9b...,117838,2019-06-17 03:46:46,...,NaN,NaN,0.077226,1.660158,NaN,NaN,0.004865,2.181354e-03,NaN,NaN
40048,525989x1709x0,525989,af14ba1556970820cd358806adfe3f739ed741dccea0bf...,bc1qaw0nxph7h3kxgng0zg8jftjsulsel35mgy5t7t77x6...,NaN,NaN,02e9fa6b495d07ce19796b06b5c0f51a401dbb48935c3a...,033401a6d1a5ac4ac54a0a45b6f87c871ec897c46b9dc9...,21600,2019-07-08 03:39:18,...,NaN,NaN,-0.011045,11.151760,NaN,NaN,0.003510,1.474100e-16,NaN,NaN


**SCRIPT: Actual current betweeness for closures**

In [53]:
# Compute actual current betweeness for channel closures

futures_bet_actclose=collection_compare(blocks,channel_closures,graph_snapshots,snapshot_bet,measurement='current_betweeness',type_dec='actual')
start=time.time()
bet_actclose_diclist = dask.compute(*futures_bet_actclose)
end=time.time()
print('Compute in seconds: {}'.format(end-start))
print('Size in memory: {}'.format(sys.getsizeof(bet_actclose_diclist)))


Compute in seconds: 7.168177366256714
Size in memory: 1640


In [56]:
# Write to marginal current betweeness for opens into decisions Dataframe

bet_actclose_channels=add_columns(bet_actclose_diclist,decisions_df,'bet_actclose_node0','bet_actclose_node1')

# Updated DataFrame

print('Rows edited with actual current betweeness for closures: {}'.format(len(bet_actclose_channels)))
decisions_df[decisions_df['short_channel_id'].isin(bet_actclose_channels)].head(2)

Rows edited with actual current betweeness for closures: 0


,short_channel_id,open_block,open_transaction,address,close_block,close_transaction,node0,node1,satoshis,last_seen,...,bet_marclose_node0,bet_marclose_node1,clo_maropen_node0,clo_maropen_node1,clo_marclose_node0,clo_marclose_node1,bet_actopen_node0,bet_actopen_node1,bet_actclose_node0,bet_actclose_node1


**SCRIPT: Actual current closeness for opens**

In [61]:
# Compute actual current closeness for channel opens

futures_clo_actopen=collection_compare(blocks,channel_opens,graph_snapshots,snapshot_bet,measurement='current_closeness',type_dec='actual')
start=time.time()
clo_actopen_diclist = dask.compute(*futures_clo_actopen)
end=time.time()
print('Compute in seconds: {}'.format(end-start))
print('Size in memory: {}'.format(sys.getsizeof(bet_actopen_diclist)))


Compute in seconds: 5.447939395904541
Size in memory: 1640


In [62]:
# Write to marginal current closeness for opens into decisions Dataframe

clo_actopen_channels=add_columns(clo_actopen_diclist,decisions_df,'clo_actopen_node0','clo_actopen_node1')

# Updated DataFrame

print('Rows edited with actual current closeness for opens: {}'.format(len(clo_actopen_channels)))
decisions_df[decisions_df['short_channel_id'].isin(clo_actopen_channels)].head(2)

Rows edited with actual current closeness for opens: 218


,short_channel_id,open_block,open_transaction,address,close_block,close_transaction,node0,node1,satoshis,last_seen,...,clo_maropen_node0,clo_maropen_node1,clo_marclose_node0,clo_marclose_node1,bet_actopen_node0,bet_actopen_node1,bet_actclose_node0,bet_actclose_node1,clo_actopen_node0,clo_actopen_node1
290,525952x80x0,525952,9e0ec787af59bdb7affc8be04c379733f94b06e52833fb...,bc1qk9trrgwz8kl6ryklxzmaky2jfgc0lycv7zcajeu82m...,578854.0,00d3ddedfb6cd6662745b912eb361fc70ed57a088c5e26...,024cd3be18617f39cf645851e3ba63f51fc13f0bb09e3b...,028705acfe269be139d536a5c7a05961cbbed3484a2c9b...,117838,2019-06-17 03:46:46,...,0.077226,1.660158,NaN,NaN,0.004865,2.181354e-03,NaN,NaN,0.004865,2.181354e-03
40048,525989x1709x0,525989,af14ba1556970820cd358806adfe3f739ed741dccea0bf...,bc1qaw0nxph7h3kxgng0zg8jftjsulsel35mgy5t7t77x6...,NaN,NaN,02e9fa6b495d07ce19796b06b5c0f51a401dbb48935c3a...,033401a6d1a5ac4ac54a0a45b6f87c871ec897c46b9dc9...,21600,2019-07-08 03:39:18,...,-0.011045,11.151760,NaN,NaN,0.003510,1.474100e-16,NaN,NaN,0.003510,1.474100e-16


**SCRIPT: Actual current closeness for closures**

In [63]:
# Compute actual current closeness for channel closures

futures_clo_actclose=collection_compare(blocks,channel_closures,graph_snapshots,snapshot_bet,measurement='current_closeness',type_dec='actual')
start=time.time()
clo_actclose_diclist = dask.compute(*futures_clo_actclose)
end=time.time()
print('Compute in seconds: {}'.format(end-start))
print('Size in memory: {}'.format(sys.getsizeof(clo_actclose_diclist)))


Compute in seconds: 6.824621200561523
Size in memory: 1640


In [64]:
# Write to marginal current closeness for opens into decisions Dataframe

clo_actclose_channels=add_columns(clo_actclose_diclist,decisions_df,'clo_actclose_node0','clo_actclose_node1')

# Updated DataFrame

print('Rows edited with actual current closeness for closures: {}'.format(len(clo_actclose_channels)))
decisions_df[decisions_df['short_channel_id'].isin(clo_actclose_channels)].head(2)

Rows edited with actual current closeness for closures: 0


,short_channel_id,open_block,open_transaction,address,close_block,close_transaction,node0,node1,satoshis,last_seen,...,clo_marclose_node0,clo_marclose_node1,bet_actopen_node0,bet_actopen_node1,bet_actclose_node0,bet_actclose_node1,clo_actopen_node0,clo_actopen_node1,clo_actclose_node0,clo_actclose_node1


# -------------------------------

### Pairwise stability 

- **Marginal betweenness (bet_mar_nodei)**: The % change between the betweenness centrality, for the node under analysis, given the graph from the previous block and the betweenness centrality of the resulting graph after enacting the decission (adding or removing a channel). Weighted current betweenness centrality is used for this measure.

> **Marginal betweenness for opens** 

**TODO**: Why is length of Dataframe longer than the number of snapshots extracted? Could it be that some channels appear more than once in dataframe?

> **Marginal betweenness for closures** 

- **Actual change in betweenness (bet_act_nodei)**: The % change between the betweenness centrality, for the node under analysis, given the graph from the previous block and the betweenness centrality of the resulting graph after enacting **all** the decissions (adding or removing a channels) in the current block. Weighted current betweenness centrality is used for this measure.

- **Marginal betweeness pairwise stability (bet_mar_pairst/open/close)**: Evaluates if given the marginal graph that results from just enacting this decission is consistent with pairwise stability, from a betweenness perspective.

In [ ]:
# MARGINAL - Add column with check for pairwise stability compatability using marginal outcomes

# OPEN - Channel is opened if both nodes gain

# Define function 
def bet_pairst_maropen(row):
    if not math.isnan(row['bet_mar_node0']):
        pairst=(row['bet_maropen_node0']>=0 and row['bet_maropen_node1']>=0)
    else:
        pairst=row['bet_maropen_node0']
    return pairst

# Apply function
decisions_df['bet_mar_pairstopen']=decisions_df.apply(bet_pairst_maropen,axis=1)

# CLOSE - Channel is closed if at least one node is better off

# Define function 
def bet_pairst_marclose(row):
    if not math.isnan(row['bet_marclose_node0']):
        pairst=(row['bet_marclose_node0']>0 or row['bet_marclose_node1']>0)
    else:
        pairst=row['bet_marclose_node0']
    return pairst

# Apply function
decisions_df['bet_mar_pairstclose']=decisions_df.apply(bet_pairst_marclose,axis=1)

In [402]:
# Test MARGINAL OPEN
decisions_df[decisions_df['bet_mar_node0'].notnull()][['bet_mar_node0','bet_mar_node1','bet_mar_pairstopen']].head()

,bet_mar_node0,bet_mar_node1,bet_mar_pairstopen
2745,-0.010145,0.001177,False
2744,0.018342,0.014761,True
40227,-0.019278,-0.006038,False
2991,0.115050,0.000216,True
2940,-0.000775,0.034945,False


In [403]:
# Test MARGINAL CLOSE
decisions_df[decisions_df['bet_marclose_node0'].notnull()][['bet_marclose_node0','bet_marclose_node1','bet_mar_pairstclose']].head()

,bet_marclose_node0,bet_marclose_node1,bet_mar_pairstclose
1027,-1.619468e-13,0.000000e+00,False
1045,1.099338e-14,0.000000e+00,True
2745,0.000000e+00,0.000000e+00,False
2744,0.000000e+00,0.000000e+00,False
40227,3.089976e-17,-1.350569e-14,True


- **Actual betweeness pairwise stability (bet_act_pairstopen/close)**: Evaluates if given the marginal graph that results from all the decisions in the block is consitend with pairwise stability, from a betweenness perspective. 

In [404]:
# ACTUAL - Add column with check for pairwise stability compatability using marginal outcomes

# OPEN - Channel is opened if both nodes gain

# Define function 
def bet_pairst_actopen(row):
    if not math.isnan(row['bet_actopen_node0']):
        pairst=(row['bet_actopen_node0']>=0 and row['bet_actopen_node1']>=0)
    else:
        pairst=row['bet_actopen_node0']
    return pairst

# Apply function
decisions_df['bet_act_pairstopen']=decisions_df.apply(bet_pairst_actopen,axis=1)

# CLOSE - Channel is closed if at least one node is better off

# Define function 
def bet_pairst_actclose(row):
    if not math.isnan(row['bet_actclose_node0']):
        pairst=(row['bet_actclose_node0']>0 or row['bet_actclose_node1']>0)
    else:
        pairst=row['bet_actclose_node0']
    return pairst

# Apply function
decisions_df['bet_act_pairstclose']=decisions_df.apply(bet_pairst_actclose,axis=1)

In [406]:
# Test ACTUAL OPEN
decisions_df[decisions_df['bet_actopen_node0'].notnull()][['bet_actopen_node0','bet_actopen_node1','bet_act_pairstopen']].head()

,bet_actopen_node0,bet_actopen_node1,bet_act_pairstopen
1027,0.000000e+00,0.000000e+00,True
1045,1.061450e-07,5.605437e-04,True
2745,-2.441083e-06,3.004141e-03,False
2744,-8.283609e-06,3.004141e-03,False
40227,3.089976e-17,-1.350569e-14,False


In [407]:
# Test ACTUAL CLOSE
decisions_df[decisions_df['bet_actclose_node0'].notnull()][['bet_actclose_node0','bet_actclose_node1','bet_act_pairstclose']].head()

,bet_actclose_node0,bet_actclose_node1,bet_act_pairstclose
1027,0.000000e+00,0.000000e+00,False
1045,1.061450e-07,5.605437e-04,True
2745,-2.441083e-06,3.004141e-03,True
2744,-8.283609e-06,3.004141e-03,True
40227,3.089976e-17,-1.350569e-14,True


In [427]:
# Save Updated DataFrame to S3

# Create S3 resource and define values
session = boto3.session.Session()
s3 = session.resource('s3')
csv_buffer = io.StringIO()

# File path and name ([extraction_id]snapshot_bet-[no_blocks]-[start_block]-[end_block])
key_decisions_df='graph_snapshots/'+str(extraction_id)+'_connected/.data_transformations/'+str(extraction_id)+'decisions_df_bet-'+str(no_blocks)+'-'+str(start_block)+'-'+str(end_block)+'.csv'


# Safe DataFrame
decisions_df.to_csv(csv_buffer)
s3.Object(bucket, key_decisions_df).put(Body=csv_buffer.getvalue())


{'ResponseMetadata': {'RequestId': '93BF4513B8A3134F',
  'HostId': '/fMvjqHAFsQUc1Y+97V5LVQuyDGpBpyZX0LY2+iD+Wh/G1ERU6up9Y7EuRQRPPBY+g67lFjTJj8=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '/fMvjqHAFsQUc1Y+97V5LVQuyDGpBpyZX0LY2+iD+Wh/G1ERU6up9Y7EuRQRPPBY+g67lFjTJj8=',
   'x-amz-request-id': '93BF4513B8A3134F',
   'date': 'Tue, 07 Apr 2020 05:26:10 GMT',
   'etag': '"33f9c7d2fc9eb4b9bcd4b5e23702d944"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"33f9c7d2fc9eb4b9bcd4b5e23702d944"'}

In [ ]:
# Test Save
decisions_df_test_load = s3.Object(bucket_name=bucket, key=key_decisions_df).get()
decisions_df_test=pd.read_csv(io.BytesIO(decisions_df_test_load['Body'].read()),index_col=0)
decisions_df_test==decisions_df

In [433]:
decisions_df_test.head()

,short_channel_id,open_block,open_transaction,address,close_block,close_transaction,node0,node1,satoshis,last_seen,...,bet_actopen_node0,bet_actopen_node1,bet_actclose_node0,bet_actclose_node1,bet_maropen_node0,bet_maropen_node1,bet_mar_pairstopen,bet_mar_pairstclose,bet_act_pairstopen,bet_act_pairstclose
1027,535029x2012x1,535029,d01928d350e1ba04d7335a91e6dd54f5dbf94859e0c59b...,bc1qszamn0la3yqrqhjj8yepdxkl9qlr84zfwgg9zrkccl...,535029.0,d01928d350e1ba04d7335a91e6dd54f5dbf94859e0c59b...,022a7809052db05fde648391a53aba82286e4a517cff1d...,031b71cbad0cb4e22141e45f16c83c332f755e1ba68195...,462124,2019-07-18 02:48:37,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.002350,4.163955e-15,NaN,False,True,False
1045,535177x446x1,535177,7376d5bc0c18bbff8f644d0827e759a1518b38e1e95a08...,bc1qauzljedtlva73ngg7suqketlvn5gnnuemxpeuevcqt...,535177.0,7376d5bc0c18bbff8f644d0827e759a1518b38e1e95a08...,02272bd12e59324d0f2b231fb88f134b57eb26dd100d2c...,031b71cbad0cb4e22141e45f16c83c332f755e1ba68195...,257307,2019-07-18 02:48:40,...,1.061450e-07,5.605437e-04,1.061450e-07,5.605437e-04,0.001134,1.120106e-03,NaN,True,True,True
2745,549037x2738x0,549037,b7128bbbe422b4f18fad71b091eed1f9e4b0d231be8117...,bc1q95fytjzs8f7fma2nf66gcva7c3w7hnkdwrkef9pu33...,549037.0,b7128bbbe422b4f18fad71b091eed1f9e4b0d231be8117...,028b892b15f5cabcea5165b236db0e36dc06553c323c84...,038b36a43c38f75cd15bb25394f1cd162f717df0055852...,400000,2019-08-22 02:59:19,...,-2.441083e-06,3.004141e-03,-2.441083e-06,3.004141e-03,0.001614,3.589994e-15,False,False,False,True
2744,549037x2737x0,549037,0825da5e96cd45fced3233ebe615721b687285839d3036...,bc1q5mqzhw5e42rfqh250zalwu47ru8gvz4g4k968me0mg...,549037.0,0825da5e96cd45fced3233ebe615721b687285839d3036...,02b95713bbe4609a337f3ca5aab3a75674083ddf5331a4...,038b36a43c38f75cd15bb25394f1cd162f717df0055852...,400000,2019-08-22 02:59:19,...,-8.283609e-06,3.004141e-03,-8.283609e-06,3.004141e-03,0.001600,2.819470e-15,True,False,False,True
40227,549489x1194x1,549489,58dafe493648fbdd69143c26e0cf8a66ae11a272c2739d...,bc1q25j5l6crv4mrjkjjjw4rzyv890cwwnyyw9dezcqs5x...,549489.0,58dafe493648fbdd69143c26e0cf8a66ae11a272c2739d...,02574ffa55d394b9326f6e5c15992cc0516b0d6e6a79a1...,03a5927b64b1ea8657d5b770d61a3e2d0554fdb5d56877...,2500000,2019-06-13 01:12:27,...,3.089976e-17,-1.350569e-14,3.089976e-17,-1.350569e-14,0.000248,2.496207e-03,False,True,False,True


In [434]:
decisions_df.head()

,short_channel_id,open_block,open_transaction,address,close_block,close_transaction,node0,node1,satoshis,last_seen,...,bet_actopen_node0,bet_actopen_node1,bet_actclose_node0,bet_actclose_node1,bet_maropen_node0,bet_maropen_node1,bet_mar_pairstopen,bet_mar_pairstclose,bet_act_pairstopen,bet_act_pairstclose
1027,535029x2012x1,535029,d01928d350e1ba04d7335a91e6dd54f5dbf94859e0c59b...,bc1qszamn0la3yqrqhjj8yepdxkl9qlr84zfwgg9zrkccl...,535029.0,d01928d350e1ba04d7335a91e6dd54f5dbf94859e0c59b...,022a7809052db05fde648391a53aba82286e4a517cff1d...,031b71cbad0cb4e22141e45f16c83c332f755e1ba68195...,462124,2019-07-18 02:48:37,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.002350,4.163955e-15,NaN,False,True,False
1045,535177x446x1,535177,7376d5bc0c18bbff8f644d0827e759a1518b38e1e95a08...,bc1qauzljedtlva73ngg7suqketlvn5gnnuemxpeuevcqt...,535177.0,7376d5bc0c18bbff8f644d0827e759a1518b38e1e95a08...,02272bd12e59324d0f2b231fb88f134b57eb26dd100d2c...,031b71cbad0cb4e22141e45f16c83c332f755e1ba68195...,257307,2019-07-18 02:48:40,...,1.061450e-07,5.605437e-04,1.061450e-07,5.605437e-04,0.001134,1.120106e-03,NaN,True,True,True
2745,549037x2738x0,549037,b7128bbbe422b4f18fad71b091eed1f9e4b0d231be8117...,bc1q95fytjzs8f7fma2nf66gcva7c3w7hnkdwrkef9pu33...,549037.0,b7128bbbe422b4f18fad71b091eed1f9e4b0d231be8117...,028b892b15f5cabcea5165b236db0e36dc06553c323c84...,038b36a43c38f75cd15bb25394f1cd162f717df0055852...,400000,2019-08-22 02:59:19,...,-2.441083e-06,3.004141e-03,-2.441083e-06,3.004141e-03,0.001614,3.589994e-15,False,False,False,True
2744,549037x2737x0,549037,0825da5e96cd45fced3233ebe615721b687285839d3036...,bc1q5mqzhw5e42rfqh250zalwu47ru8gvz4g4k968me0mg...,549037.0,0825da5e96cd45fced3233ebe615721b687285839d3036...,02b95713bbe4609a337f3ca5aab3a75674083ddf5331a4...,038b36a43c38f75cd15bb25394f1cd162f717df0055852...,400000,2019-08-22 02:59:19,...,-8.283609e-06,3.004141e-03,-8.283609e-06,3.004141e-03,0.001600,2.819470e-15,True,False,False,True
40227,549489x1194x1,549489,58dafe493648fbdd69143c26e0cf8a66ae11a272c2739d...,bc1q25j5l6crv4mrjkjjjw4rzyv890cwwnyyw9dezcqs5x...,549489.0,58dafe493648fbdd69143c26e0cf8a66ae11a272c2739d...,02574ffa55d394b9326f6e5c15992cc0516b0d6e6a79a1...,03a5927b64b1ea8657d5b770d61a3e2d0554fdb5d56877...,2500000,2019-06-13 01:12:27,...,3.089976e-17,-1.350569e-14,3.089976e-17,-1.350569e-14,0.000248,2.496207e-03,False,True,False,True


### Efficiency
- **Average betweeness per block (bet_effic)**: Average betweenness centrality for all the nodes. 

### Nash stability 

- **% Change with respect to not making decision (bet_binstat_deltai)**: The % change in betwewnness centrality, for the node under analysis, given the resulting graph after all of the decissions have been executed. 
- **Nash compatible - binary strategy (bet_binstat_nash)**: Returns true if given the other decissions enacted in the block not making decision would have NOT have resulted in higher betweenness centrality. This tells me if my strategy helped me be better off (took into account what others were doing)

(Optional approaches - Check for tracktability)
- **Nash compatible - close only strategy (bet_closestat_nash)**: Returns true if given the other decissions enacted in the block, closing any other channels would NOT have not resulted in higher betwneenness centrality. (NOTE: Check if there are combinatorial considerations, if so just look at closings up to x) 
- **Nash compatible - close/open (bet_allstat_nash)**: Returns true if given the other decissions enacted in the block, closing any other channels (with any node) or opening a channel with one of the round participants would NOT have not resulted in lower betwneenness centrality. (NOTE: To make it reasonable and constraint the strategy space only consider 'similar nodes' or with relationships in the past?).






## Connectivity

### Pairwise stability 

- **Marginal % change in connectivity (con_mar_deltai)**: The % change between the shortest path average, for the node under analysis, given the graph from the previous block and the shortest path average of the resulting graph after enacting the decission (adding or removing a channel). Weighted shortest path (_single_source_dijkstra_path_) is used for this measure.

- **Actual % change in connectivity (con_act_deltai)**: The % change between the shortest path average, for the node under analysis, given the graph from the previous block and the shortest path average of the resulting graph after enacting **all** the decissions (adding or removing a channels) in the current block. Weighted shortest path (_single_source_dijkstra_path_) is used for this measure.

- **Marginal connectivity pairwise stability (con_mar_pairstab)**: Evaluates if given the marginal graph that results from just enacting this decission is consistent with pairwise stability, from a connectivity perspective.

- **Actual connectivity pairwise stability (con_act_pairstab)**: Evaluates if given the marginal graph that results from all the decisions in the block is consitend with pairwise stability, from a connectivity perspective.  



### Nash stability 

- **% Change with respect to not making decision (con_binstat_deltai)**: The % change in shortest path average, for the node under analysis, given the resulting graph after all of the decissions have been executed. 
- **Nash compatible - binary strategy (con_binstat_nash)**: Returns true if given the other decissions enacted in the block not making decision would have NOT have resulted in higher shortest path average. NOTE: This indicates if the strategy selected made the node better off (took into account what others were doing)

(Optional approaches - Check for tracktability)
- **Nash compatible - close only strategy (con_closestat_nash)**: Returns true if given the other decissions enacted in the block, closing any other channels would NOT have not resulted in higher shortest path average. (NOTE: Check if there are combinatorial considerations, if so just look at closings up to x) 
- **Nash compatible - close/open (con_allstat_nash)**: Returns true if given the other decissions enacted in the block, closing any other channels (with any node) or opening a channel with one of the round participants would NOT have not resulted in lower shortest path average. (NOTE: To make it reasonable and constraint the strategy space only consider 'similar nodes' or with relationships in the past?).



### Efficiency
- **Average betweeness per block (bet_effic)**: Average shortest path average for all the nodes. 


## Utility Functions

In [93]:
def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))